In [1]:
import cv2
import numpy as np

"""
    전에 노란 차선을 직접 추출해보셨을 겁니다.
    그럼 이제 전에 배운 모든걸 동원하여 도로 영상에서
    노란 차선과 흰 차선을 찾아서 허프 변환을 이용해 선분 좌표를 구하여
    이를 원본 영상에 파란색 선으로 그려주세요.
"""

def nothing(x):
    pass

def region_of_interest(img, roi):
    vertices = np.array([roi], np.int32)
    mask = np.zeros_like(img)

    # channel_count = img.shape[2]
    # match_mask_color = (255,) * channel_count
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def run():
    video = cv2.VideoCapture("image/road2.mp4")

    ret, _image = video.read()
    #_image = cv2.imread("../img/road_00.JPG")
    height = _image.shape[0]
    width = _image.shape[1]

    hsv = cv2.cvtColor(_image, cv2.COLOR_BGR2HSV)

    roi = [
        (width/7, height-20),
        (width * 3 / 7, height * 3 / 4),
        (width * 4 / 7, height * 3 / 4),
        (width*6/7, height-20)
    ]

    create_control_window()

    while ret:
        hsv = cv2.cvtColor(_image, cv2.COLOR_BGR2HSV)
        high_val, low_val = get_control_value()
        mask = cv2.inRange(hsv, low_val, high_val)
        mask = region_of_interest(mask, roi)

        edges = cv2.Canny(mask, 75, 150)
        cv2.imshow("Canny", edges)
        rho = 1  # distance precision in pixel
        angle = np.pi / 180
        min_threshold = 60

        line_segments = cv2.HoughLinesP(edges, rho, angle, min_threshold,
                                        np.array([]), minLineLength=100, maxLineGap=400)
        try:
            for i in line_segments:
                for j in i:
                    cv2.line(_image, (j[0], j[1]), (j[2], j[3]), (255, 0, 0), 5)
        except:
            pass
        cv2.imshow("Result", _image)

        if cv2.waitKey(20) & 0xff == ord('q'):
            break

        ret, _image = video.read()


    cv2.destroyAllWindows()


def create_control_window():
    cv2.namedWindow('Control')
    cv2.createTrackbar('h_max', 'Control', 0, 180, nothing)
    cv2.createTrackbar('h_min', 'Control', 0, 180, nothing)
    cv2.createTrackbar('s_max', 'Control', 0, 255, nothing)
    cv2.createTrackbar('s_min', 'Control', 0, 255, nothing)
    cv2.createTrackbar('v_max', 'Control', 0, 255, nothing)
    cv2.createTrackbar('v_min', 'Control', 0, 255, nothing)

    cv2.setTrackbarPos('h_max', 'Control', 125)
    cv2.setTrackbarPos('h_min', 'Control', 0)
    cv2.setTrackbarPos('s_max', 'Control', 255)
    cv2.setTrackbarPos('s_min', 'Control', 0)
    cv2.setTrackbarPos('v_max', 'Control', 255)
    cv2.setTrackbarPos('v_min', 'Control', 159)

def get_control_value():
    h_min = cv2.getTrackbarPos('h_min', 'Control')
    h_max = cv2.getTrackbarPos('h_max', 'Control')
    s_min = cv2.getTrackbarPos('s_min', 'Control')
    s_max = cv2.getTrackbarPos('s_max', 'Control')
    v_min = cv2.getTrackbarPos('v_min', 'Control')
    v_max = cv2.getTrackbarPos('v_max', 'Control')

    high_val = (h_max, s_max, v_max)
    low_val = (h_min, s_min, v_min)

    return high_val, low_val


if __name__ == "__main__":
    run()

AttributeError: 'NoneType' object has no attribute 'shape'